In [1]:
#Notebook for experiments in gathering data from https://deadline.com/, etc

In [2]:
import re
import requests
from utils import parse_tools

In [9]:
class DeadlineParseFormat:
    host_name = "https://deadline.com"
    categories = ["film"]
    min_page_num = 1
    max_page_num = 100000 #inf
    def hay_page_URL(self, cat_name, page_num):
        return f"{self.host_name}/v/{cat_name}/page/{page_num}"
    def needle_URL_regexp(self, cat_name=''):
        return f"^{self.host_name}/\d*/\d*/[-\w]*"

In [10]:
site_format = DeadlineParseFormat()

In [11]:
urls = set()
hay_url = site_format.hay_page_URL(cat_name="film", page_num=2)
response = requests.get(hay_url)

In [15]:
urls = set()            
needle_pattern = site_format.needle_URL_regexp()
parse_tools.gather_page_urls(response.text, needle_pattern, urls)            

In [16]:
result = re.match(needle_pattern, 'https://deadline.com/2019/04/deadline-relaunch-poll-redesign-1202592366/#comments')
result[0]

'https://deadline.com/2019/04/deadline-relaunch-poll-redesign-1202592366'

In [17]:
print(len(urls))
print(urls)

22
{'https://deadline.com/2019/04/ethan-hawke-tony-watch-interview-true-west-good-lord-bird-1202592003', 'https://deadline.com/2019/04/deadline-relaunch-poll-redesign-1202592366', 'https://deadline.com/2019/04/the-lion-king-trailer-disney-jon-favreau-circle-of-life-chiwetel-ejifor-1202592616', 'https://deadline.com/2019/04/filmnation-podcast-division-launch-1202592793', 'https://deadline.com/2019/04/the-dropout-hulu-limited-series-kate-mckinnon-star-elizabeth-holmes-fox-searchlight-television-abc-news-1202593032', 'https://deadline.com/2019/04/the-plot-against-america-winona-ryder-zoe-kazan-morgan-spector-john-turturro-hbo-miniseries-1202592810', 'https://deadline.com/2019/04/riverdale-camila-mendes-netflix-windfall-movie-palm-springs-1202591378', 'https://deadline.com/2019/04/black-widow-marvel-cast-handmaids-tale-actor-o-t-fagbenle-1202593084', 'https://deadline.com/2019/04/avengers-endgame-agbo-joe-russo-anthony-russo-mgm-library-the-thomas-crown-affair-1202591932', 'https://deadlin